In [1]:
import glob 
import pandas as pd
import os
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup as BS 
from tqdm import tqdm
import folium
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc 
import matplotlib.font_manager as fm
import seaborn as sns
import pymysql
import warnings
warnings.filterwarnings('ignore')
# csv_test = pd.read_csv('./cycle.csv')
geo_path = './cycle_road.geojson'



In [2]:
con =pymysql.connect(host='ip', user='id', password='pw', db='db', charset='utf8')
cur = con.cursor()
#사고위반유형별 월별 자전거 사고 데이터
violation_accident_cycle_month = pd.read_sql_query("select * from violation_accident_cycle_month", con=con)

In [3]:
violation_accident_cycle_month.head(1)

,JIJACE_NM,CHISA,ACC_HUM,SIDO_CD,MONTH,SLTWD_CNT,SERI_ACC,SIDO_NM,AC_MODE,CRT_JIJACE_CD,...,TRK_CNT,DEATH_CNT,BASE_CODE,WND_CNT,DEATH_ACC,GUBUN,SERI_CNT,SLTWD_ACC,YEAR,BASE_CATEGORY
0,강남구,0.0,2,11000,02,2,0,서울,A,11680,...,0,0,01,0,0,02,0,1,2018,위반


In [4]:
violation_accident_cycle_month_group = violation_accident_cycle_month.groupby(["YEAR","MONTH","JIJACE_NM"],as_index=False)[['ACC_CNT']].sum()

In [5]:
violation_accident_cycle_month_group.head(5)
#자전거 년별 사고 확인했고 지자체확인하자

,YEAR,MONTH,JIJACE_NM,ACC_CNT
0,2018,01,강남구,1
1,2018,01,강동구,4
2,2018,01,강서구,3
3,2018,01,관악구,3
4,2018,01,광진구,4


In [6]:
violation_accident_cycle_month_group_2020 = violation_accident_cycle_month_group[violation_accident_cycle_month_group.YEAR<=2020]

In [7]:
geo_road = json.load(open(geo_path, encoding='utf-8'))
map= folium.Map(location=[37.5502, 126.982], zoom_start=11,tiles="cartodb positron")
folium.Choropleth(geo_data=geo_road,
                  line_color='black',
                  line_weight=1.5,
                  line_opacity=0.5,
                  key_on='feature.id').add_to(map)
# map


In [8]:
cycle_where = pd.read_excel('./cycle_where.xlsx')

In [9]:
cycle_where.다발지구분.unique()

array(['2018년 다발지', '2019년 다발지', '2020년 다발지'], dtype=object)

In [10]:
cycle_where.head(1)

,다발지구분,다발지시군구,지점명,발생일,발생시간대,사고내용,가해자성별,가해자연령,가해자차종,가해자신체상해정도,피해자성별,피해자연령,피해자차종,피해자신체상해정도,사고유형,법규위반사항,기상상태,사망자수,부상자수
0,2018년 다발지,서울특별시 종로구1,서울특별시 종로구 종로6가(흥인지문사거리 부근),2018-03-23,07시,경상,남,59세,자전거,상해없음,남,40세,보행자,경상,차대사람 - 횡단중,신호위반,맑음,0,5


In [11]:
from geopy.geocoders import Nominatim
import osmnx as ox 
geolocoder = Nominatim(user_agent = 'South Korea')
def geocoding(address): 
    # address="\""+address+"\""
    geo = geolocoder.geocode(address)
    crd = (geo.latitude, geo.longitude)
    # print(crd)
    return crd

In [6]:
# cycle_where.dropna()

In [13]:
geocoding("서울특별시 종로구 종로6가")

(37.57174, 127.00681)

In [14]:
geocoding("경상남도 거창군 거창읍")

(35.6863036, 127.9100451)

In [15]:
cycle_where = cycle_where[cycle_where.다발지시군구.str.find("서울특별시")>-1]

In [16]:
type(cycle_where.iloc[0].지점명)

str

In [17]:
where_list=[]


In [26]:
for idx in tqdm(range(len(cycle_where))): #series형태로들어감
   
    # print(type(geocoding(address)))
    # print(geocoding(address)[0])
    try:
        address=cycle_where.iloc[idx].지점명.split("(")[0]
        temp=list(geocoding(address))
        where_list.append(temp)
    except:
        print(idx)
        # pass
    
    
    # print(cycle_where.loc[idx,"지점명"])
    # try:
    #     geocoding(cycle_where.loc[idx,"지점명"])
    # except:
    #     print(idx)
    # 1046 1047 1048 1049 1050 1051 데이터없음
    # break
    # pass
    # folium.Marker([row.lat,row.lot],popup=row.s_name).add_to(map)

 51%|████████████████████████████████████████▏                                      | 1047/2057 [08:45<08:22,  2.01it/s]

1046


 51%|████████████████████████████████████████▏                                      | 1048/2057 [08:46<08:34,  1.96it/s]

1047


 51%|████████████████████████████████████████▎                                      | 1049/2057 [08:46<08:24,  2.00it/s]

...

 51%|████████████████████████████████████████▎                                      | 1051/2057 [08:47<08:21,  2.01it/s]

1050


 51%|████████████████████████████████████████▍                                      | 1052/2057 [08:48<08:38,  1.94it/s]

1051


100%|███████████████████████████████████████████████████████████████████████████████| 2057/2057 [17:12<00:00,  1.99it/s]


In [29]:
where_list = where_list[1:]

In [33]:
for i in range(len(where_list)):
    folium.Circle(where_list[i],radius = 50,
        color = '#FF0000',
        fill = 'crimson').add_to(map)

In [ ]:
folium.Circle([geocoding(address)[0],geocoding(address)[1]],radius = 50,
        color = '#FF0000',
        # fill = 'crimson').add_to(map)

In [5]:
# map

In [78]:
cycle_road = pd.read_csv('./riding_road.csv')

In [4]:
# cycle_road

In [42]:
cycle_road.columns

Index(['지역별(1)', '지역별(2)', '지역별(3)', '2022', '2022.1', '2022.2', '2022.3',
       '2022.4', '2022.5', '2022.6', '2022.7', '2022.8', '2022.9'],
      dtype='object')

In [74]:
# real_gugun_list = cycle_road["지역별(3)"][5:30]

In [75]:
# real_gugun_list

In [76]:
# len(cycle_road)

In [86]:
cycle_road_1=cycle_road[5:30]

In [100]:
cycle_road_1.columns

Index(['지역별(1)', '지역별(2)', '지역별(3)', '2022', '2022.1', '2022.2', '2022.3',
       '2022.4', '2022.5', '2022.6', '2022.7', '2022.8', '2022.9'],
      dtype='object')

In [106]:
cycle_road_2 = cycle_road_1["지역별(3)"]

In [108]:
# len(cycle_road_2) #25

In [109]:
cycle_road_3 = cycle_road_1["2022"]

In [111]:
# len(cycle_road_3)

In [112]:
new_cycle = pd.concat([cycle_road_2,cycle_road_3],axis=1)

In [126]:
new_cycle.rename(columns={"지역별(3)" : "지역", "2022" : "value"},inplace=True)

In [142]:
type(new_cycle.iloc[1][1])

str

In [144]:
new_cycle.value=int(new_cycle.value)

TypeError: cannot convert the series to <class 'int'>

In [160]:
for i in range(len(new_cycle)):
    a=int(new_cycle.iloc[i][1])
    b=int(a)
    # print(b)
    new_cycle.iloc[i]["count"]=b
    # print(b)


In [129]:
type(new_cycle.value)

pandas.core.series.Series

In [3]:
# new_cycle

In [99]:
# print(cycle_road_1.iloc[2]["지역별(3)"])

In [119]:
# for i in range(len(cycle_road_1)):
    # print(cycle_road_1.iloc[i]["지역별(3)"])
    # if cycle_road.iloc[i,"지역별(3)"] in real_gugun_list:
    #     print("1")

In [120]:
geo_str = requests.get("https://t1.daumcdn.net/cfile/tistory/272C224C58B4BD540B").json()

In [2]:
# geo_str

In [136]:
m = folium.Choropleth(geo_str, 
                  data = new_cycle,
                  columns=['지역', 'value'],
                  fill_color="PuRd",
                  key_on="feature.id").add_to(map)
                  

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [1]:
# map